In [ ]:
import numpy as np
import os
import glob

# Defining the root folder containing wavelet results
root_folder = 'wavelet_results' 

# List of sub-band files to process (for MAV ratio calculations)
subbands = ['D1.txt', 'D2.txt', 'D3.txt', 'D4.txt', 'D5.txt', 'A5.txt']

# Process both A_extended and E_extended folders (normal and epileptic patients)
for data_type in ['A_extended', 'E_extended']:
    label = 1 if data_type == 'A_extended' else 0 
    data_folder = os.path.join(root_folder, data_type)

    # Looping through each EEG file's folder
    folders = glob.glob(os.path.join(data_folder, '*'))

    for folder in folders:
        filename = os.path.basename(folder)  # Get the filename from the folder path

        previous_mean = None  # Store the MAV of the previous sub-band for ratio calculations

        # Create a folder to store results for the current EEG file
        results_folder = os.path.join(folder, 'calculated_metrics')
        os.makedirs(results_folder, exist_ok=True)  

        # Processing each sub-band
        for subband in subbands:
            subband_path = os.path.join(folder, subband)

            if os.path.exists(subband_path):
                
                coeffs = np.loadtxt(subband_path) # coefficients for the current sub-band

                # Calculating metrics
                mav = np.mean(np.abs(coeffs))  # Mean of absolute values (MAV)
                avg_power = np.mean(coeffs ** 2)  # Average power
                std_dev = np.std(coeffs)  # Standard deviation

                # Calculating ratio of MAVs between adjacent sub-bands
                ratio = mav / previous_mean if previous_mean is not None else np.nan
                previous_mean = mav  # Updating previous_mean for the next sub-band

                # Creating a subfolder for the current sub-band (e.g., D1)
                subband_folder = os.path.join(results_folder, subband.replace('.txt', ''))
                os.makedirs(subband_folder, exist_ok=True)  

                # Saving metrics to a text file inside the sub-band folder
                metrics_file = os.path.join(subband_folder, 'metrics.txt')
                with open(metrics_file, 'w') as f:
                    f.write(f"Label: {label}\n")  
                    f.write(f"MAV: {mav}\n")
                    f.write(f"Average Power: {avg_power}\n")
                    f.write(f"Standard Deviation: {std_dev}\n")
                    f.write(f"MAV Ratio: {ratio}\n")
            else:
                print(f"Sub-band file {subband} not found in {folder}.")
